In [15]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import feature_engineering
import model_xgboost
importlib.reload(model_xgboost)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

# Experimento 7: 
- LGBM
- Usando funcion entrenamiento: semillerio_en_prediccion
- Aumento cantidad de variables
- sqlite:///optuna_studies_v15.db
- Kaggle =  


##### Levantamos el dataset con target ya calculado

In [16]:
df = pd.read_csv("../../data/preprocessed/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

In [18]:
columnas_baseline = df.columns.tolist()
columnas_baseline

['product_id',
 'periodo',
 'nacimiento_producto',
 'muerte_producto',
 'mes_n',
 'total_meses',
 'producto_nuevo',
 'ciclo_de_vida_inicial',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'sku_size',
 'stock_final',
 'tn',
 'plan_precios_cuidados',
 'cust_request_qty',
 'cust_request_tn',
 'target']

##### Preprocesamiento a la minima expresión :)

In [50]:
# ##### aplicamos OHE
# df = preprocesamiento.aplicarOHE(df)
# df.shape

### Feature Engineering

##### Neural Prophet

In [19]:
neural_prophet_fe = pd.read_csv("../../data/preprocessed/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 22)

##### Prophet

In [20]:
prophet_fe = pd.read_csv("../../data/preprocessed/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 28)

##### FE Moviles

In [21]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 590)

In [22]:
df = feature_engineering.get_lags(df, "cust_request_qty", 201912)
df = feature_engineering.get_delta_lags(df, "cust_request_qty", 24)
df = feature_engineering.get_rolling_means(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_stds(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_mins(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_maxs(df, "cust_request_qty", 201912)

In [23]:
df = feature_engineering.get_lags(df, "stock_final", 201912)
df = feature_engineering.get_delta_lags(df, "stock_final", 24)
df = feature_engineering.get_rolling_means(df, "stock_final", 201912)
df = feature_engineering.get_rolling_stds(df, "stock_final", 201912)
df = feature_engineering.get_rolling_mins(df, "stock_final", 201912)
df = feature_engineering.get_rolling_maxs(df, "stock_final", 201912)
df.shape

(31362, 1500)

Features Diana

In [24]:
df = feature_engineering.calcular_diferencia_con_medias_moviles(df)
df = feature_engineering.calcular_ratios_con_medias_moviles(df)

##### FE Moviles sobre otras variables

In [56]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

#  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty')

##### FE Calendario

In [25]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 1561)

##### FE sobre FE

In [26]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 1590)

##### Variables Exogenas

In [27]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 1593)

##### Nuevas FE

In [28]:
df = feature_engineering.create_ratio_features(df)
df = feature_engineering.enhance_lifecycle_features(df)
df = feature_engineering.create_category_features(df)
df = feature_engineering.create_regime_features(df)
df = feature_engineering.create_nonlinear_trends(df)
df = feature_engineering.create_temporal_interactions(df)
df = feature_engineering.create_asymmetric_window_features(df)
df = feature_engineering.recomendaciones_deepseek(df)
df = feature_engineering.get_nuevas_features(df)
df.shape

(31362, 1635)

##### Elimino aquellas que no sirven

In [ ]:
# importantes = pd.read_csv("./feature_importance/exp04_3.csv", sep=',', encoding='utf-8')
# no_importantes = importantes[importantes['importance'] <= 100]
# no_importantes = no_importantes[~no_importantes['feature'].isin(columnas_baseline)]
# no_importantes

,feature,importance
67,tn_vs_prev_year,100
68,tn_delta_lag5_lag8,99
69,tn_rolling_min_4,96
70,tn_lag_5,95
71,tn_rolling_std_6,95
...,...,...
677,dia_del_year,0
678,cat2_TE,0
680,cat2_PIEL1,0
681,cat2_OTROS,0


In [ ]:
# cols_a_eliminar = no_importantes.feature.unique()
# print(f"Antes de eliminar: {df.shape[1]} columnas")
# df = df.drop(columns=cols_a_eliminar, errors='ignore')
# print(f"Después de eliminar: {df.shape[1]} columnas")

Antes de eliminar: 1144 columnas
Después de eliminar: 684 columnas


Eliminar object/categorical columnas

In [30]:
df = df.select_dtypes(exclude=['datetime', 'datetime64', 'object','category'])
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)  # o usá otra imputación según tu criterio
df = df.clip(lower=-1e6, upper=1e6)  # evita valores descomunales

Train Test Split

In [31]:
train = df[df['periodo'] <= 201912]
test = df[df['periodo'] == 201912]

Entrenamiento

In [ ]:
model_xgboost.optimizar_con_optuna_xgboost_db(train, version="v4")

[I 2025-07-03 15:08:21,109] Using an existing study with name 'xgboost_optimization_v3' instead of creating a new one.



Para visualizar los resultados en tiempo real:
1. Abre otra terminal y ejecuta:
   optuna-dashboard sqlite:///optuna_studies_v3.db
2. Abre en tu navegador: http://127.0.0.1:8080/
[15:08:22] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31356, 1629, 51078924).
[15:08:22] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 1629, 1629).
[15:12:11] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31357, 1629, 51080553).
[15:12:13] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 1629, 1629).
[15:15:51] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31358, 1629, 51082182).
[15:15:51] INFO: C:\action

[I 2025-07-03 15:26:27,578] Trial 4 finished with value: 0.9771856278181076 and parameters: {'learning_rate': 0.060829351443937155, 'n_estimators': 598, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.6277044963040949, 'colsample_bytree': 0.774136440649611, 'lambda': 3.2434812919138998, 'alpha': 4.598219036381456e-05, 'max_bin': 448}. Best is trial 4 with value: 0.9771856278181076.


Mejor trial hasta ahora: RMSE=0.977186, Parámetros={'learning_rate': 0.060829351443937155, 'n_estimators': 598, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.6277044963040949, 'colsample_bytree': 0.774136440649611, 'lambda': 3.2434812919138998, 'alpha': 4.598219036381456e-05, 'max_bin': 448}
[15:26:28] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31356, 1629, 51078924).
[15:26:29] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 1629, 1629).
[15:27:26] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31357, 1629, 51080553).
[15:27:26] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 1629, 1629).
[15:28:23] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\i

[I 2025-07-03 15:31:13,613] Trial 5 finished with value: 0.9648760542273521 and parameters: {'learning_rate': 0.2887935482704974, 'n_estimators': 932, 'max_depth': 4, 'min_child_weight': 9, 'subsample': 0.5594163015468702, 'colsample_bytree': 0.7674650752962264, 'lambda': 1.565935298822263e-05, 'alpha': 1.5373022992662747e-07, 'max_bin': 191}. Best is trial 5 with value: 0.9648760542273521.


...

[I 2025-07-03 17:51:36,014] Trial 24 finished with value: 4.499547791481018 and parameters: {'learning_rate': 0.04815699786227146, 'n_estimators': 435, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.8049356610571741, 'colsample_bytree': 0.735957575354903, 'lambda': 2.1098004973423335e-07, 'alpha': 0.0028105981075198278, 'max_bin': 254}. Best is trial 6 with value: 0.750610426068306.


Mejor trial hasta ahora: RMSE=0.750610, Parámetros={'learning_rate': 0.0371419564700273, 'n_estimators': 804, 'max_depth': 6, 'min_child_weight': 4, 'subsample': 0.9987490232425333, 'colsample_bytree': 0.6740329931778726, 'lambda': 1.1011206801452629e-05, 'alpha': 1.8921624289657625e-05, 'max_bin': 386}
[17:51:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31356, 1629, 51078924).
[17:51:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 1629, 1629).
[17:55:15] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31357, 1629, 51080553).
[17:55:15] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 1629, 1629).
[17:58:52] INFO: C:\actions-runner\_work\xgboost\xgboost\src\da

[I 2025-07-03 18:09:43,348] Trial 25 finished with value: 1.0563116170465947 and parameters: {'learning_rate': 0.06030693166553502, 'n_estimators': 613, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.6303959316228491, 'colsample_bytree': 0.7879147458605417, 'lambda': 0.5507781257047645, 'alpha': 0.00015759289647252075, 'max_bin': 438}. Best is trial 6 with value: 0.750610426068306.


Mejor trial hasta ahora: RMSE=0.750610, Parámetros={'learning_rate': 0.0371419564700273, 'n_estimators': 804, 'max_depth': 6, 'min_child_weight': 4, 'subsample': 0.9987490232425333, 'colsample_bytree': 0.6740329931778726, 'lambda': 1.1011206801452629e-05, 'alpha': 1.8921624289657625e-05, 'max_bin': 386}

Mejores hiperparámetros encontrados:
learning_rate: 0.0371419564700273
n_estimators: 804
max_depth: 6
min_child_weight: 4
subsample: 0.9987490232425333
colsample_bytree: 0.6740329931778726
lambda: 1.1011206801452629e-05
alpha: 1.8921624289657625e-05
max_bin: 386


(<optuna.study.study.Study at 0x21de0d33c50>,
 {'learning_rate': 0.0371419564700273,
  'n_estimators': 804,
  'max_depth': 6,
  'min_child_weight': 4,
  'subsample': 0.9987490232425333,
  'colsample_bytree': 0.6740329931778726,
  'lambda': 1.1011206801452629e-05,
  'alpha': 1.8921624289657625e-05,
  'max_bin': 386,
  'objective': 'reg:squarederror',
  'eval_metric': 'rmse',
  'booster': 'gbtree',
  'tree_method': 'hist',
  'verbosity': 0})

Prediccion

In [ ]:
result_df = model_xgboost.semillerio_en_prediccion_con_pesos_xgb(train, test, version="v4")

[0]	validation_0-rmse:114.64592
[1]	validation_0-rmse:110.68765
[2]	validation_0-rmse:106.87416
[3]	validation_0-rmse:103.21501
[4]	validation_0-rmse:99.71881
[5]	validation_0-rmse:96.33850
[6]	validation_0-rmse:93.11708
[7]	validation_0-rmse:90.02342
[8]	validation_0-rmse:87.04790
[9]	validation_0-rmse:84.20091
[10]	validation_0-rmse:81.46472
[11]	validation_0-rmse:78.84480
[12]	validation_0-rmse:76.33561
[13]	validation_0-rmse:73.93290
[14]	validation_0-rmse:71.64252
[15]	validation_0-rmse:69.42800
[16]	validation_0-rmse:67.31950
[17]	validation_0-rmse:65.30741
[18]	validation_0-rmse:63.37491
[19]	validation_0-rmse:61.53182
[20]	validation_0-rmse:59.77123
[21]	validation_0-rmse:58.07373
[22]	validation_0-rmse:56.46313
[23]	validation_0-rmse:54.92449
[24]	validation_0-rmse:53.47624
[25]	validation_0-rmse:52.09122
[26]	validation_0-rmse:50.77526
[27]	validation_0-rmse:49.50872
[28]	validation_0-rmse:48.30973
[29]	validation_0-rmse:47.17124
[30]	validation_0-rmse:46.07414
[31]	validatio

In [33]:
def get_promedio_780():    
    df = pd.read_csv("../../data/preprocessed/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
    df = df[df['periodo'] >= 201901]  # Filtrar desde 201901

    productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")

    df = df.merge(productos_ok, on='product_id', how='inner')

    df = df.groupby('product_id').agg({'tn': 'mean'}).reset_index()

    return df
promedios = get_promedio_780()
promedios

,product_id,tn
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104
...,...,...
775,21263,0.029993
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [34]:
result_df[result_df['pred'] < 0]

,periodo,product_id,target,pred
31029,201912,20674,0.0,-0.437661
31077,201912,20743,0.0,-0.206311
31078,201912,20744,0.0,-0.120919
31085,201912,20756,0.0,-1.054266
31086,201912,20757,0.0,-0.475207
...,...,...,...,...
31355,201912,21263,0.0,-1.602734
31356,201912,21264,0.0,-0.893278
31358,201912,21266,0.0,-0.251306
31359,201912,21267,0.0,-1.378996


In [35]:
result_df = result_df.merge(promedios, how='left', on='product_id')
result_df

,periodo,product_id,target,pred,tn
0,201912,20001,0.0,1399.183350,1454.732720
1,201912,20002,0.0,988.647949,1175.437142
2,201912,20003,0.0,627.646301,784.976407
3,201912,20004,0.0,495.018951,627.215328
4,201912,20005,0.0,538.744873,668.270104
...,...,...,...,...,...
881,201912,21265,0.0,1.404513,0.089541
882,201912,21266,0.0,-0.251306,0.094659
883,201912,21267,0.0,-1.378996,0.092835
884,201912,21271,0.0,-0.279550,NaN


In [36]:
result_df.loc[result_df['pred'] < 0 , 'pred'] = result_df['tn']
result_df

,periodo,product_id,target,pred,tn
0,201912,20001,0.0,1399.183350,1454.732720
1,201912,20002,0.0,988.647949,1175.437142
2,201912,20003,0.0,627.646301,784.976407
3,201912,20004,0.0,495.018951,627.215328
4,201912,20005,0.0,538.744873,668.270104
...,...,...,...,...,...
881,201912,21265,0.0,1.404513,0.089541
882,201912,21266,0.0,0.094659,0.094659
883,201912,21267,0.0,0.092835,0.092835
884,201912,21271,0.0,NaN,NaN


In [37]:
result_df.dropna(inplace=True)
result_df

,periodo,product_id,target,pred,tn
0,201912,20001,0.0,1399.183350,1454.732720
1,201912,20002,0.0,988.647949,1175.437142
2,201912,20003,0.0,627.646301,784.976407
3,201912,20004,0.0,495.018951,627.215328
4,201912,20005,0.0,538.744873,668.270104
...,...,...,...,...,...
879,201912,21263,0.0,0.029993,0.029993
881,201912,21265,0.0,1.404513,0.089541
882,201912,21266,0.0,0.094659,0.094659
883,201912,21267,0.0,0.092835,0.092835


Guardamos el archivo

In [38]:
result_df.drop(columns=['tn'], inplace=True)
result_df.rename(columns={'pred':'tn'}, inplace=True)
result_df[['product_id','tn']].to_csv("./predicciones_xgboost_laburo_v3.csv", sep=',', index=False)